[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ds-reboot/python-first-part/blob/main/notebooks/hometasks/Task1_dmrf.ipynb)

# Задание
* Собрать информацию о всех строящихся объектах на сайте "наш.дом.рф"
* Cохранить ее в pandas dataframe
* Cохранить pandas dataframe в excel
* Cохранить pandas dataframe в pickle
* Cохранить pandas dataframe в БД

# Задание *
* Написать скрипт, который скачивает фотографии всех строящихся объектов с сайта "наш.дом.рф"


In [1]:
import requests
import os
import pickle
import math
import hashlib
import json
import pandas as pd
import sqlite3

from tqdm.auto import tqdm
from collections import defaultdict

## Список объектов

> Загружаемые об объектах данные кэшируем на диске в папку `./data`, чтобы постоянно не загружать данные с сайта.

In [2]:
BASE_DIR = "data"

# Для хранения временных файлов.
os.makedirs("data/objects", exist_ok=True)

### Скачать ID всех объектов

In [3]:
object_ids = set()  # Только уникальные ID объектов.

url = 'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object'
url_params = {
    "offset": 0,
    "limit": 0,
    "sortField": "devId.devShortCleanNm",
    "sortType": "asc",
    "objStatus": 0,
}

# Файл содержащий ID объектов.
object_ids_file_path = f"{BASE_DIR}/object_ids.pkl"

if not os.path.isfile(object_ids_file_path):
    # Прочитать общее количество объектов.
    url_params["offset"] = 0
    url_params["limit"] = 1
    res = requests.get(url, params=url_params)
    objects_data = res.json()
    objects_total = objects_data.get("data").get("total")
    print("total:", objects_total)
  
    # Чтение всех объектов и сохранение всех ID объектов в файл,
    # чтобы повторно не вычитывать данные из сети.
    objects_offset = 0
    objects_limit = 100
    
    # Количество URL содержащих objects_limit объектов.
    iterations_count = math.ceil(objects_total/objects_limit)

    for i in tqdm(range(iterations_count)):
        # Скачать данные.
        url_params["offset"] = objects_offset
        url_params["limit"] = objects_limit
        res = requests.get(url, params=url_params)

        if 200 != res.status_code:
            break

        objects_data = res.json()
        objects_total = objects_data.get("data").get("total")
        objects_list = objects_data.get("data").get("list")
        object_ids.update([x.get("objId") for x in objects_list])

        objects_offset += objects_limit

    # Сохранить ID всех объектов в файл.
    with open(object_ids_file_path, "wb") as f:
        pickle.dump(object_ids, f)
else:
    # Файл содержащий ID всех объектов уже есть.
    with open(object_ids_file_path, "rb") as f:
        object_ids = pickle.load(f)

print("total IDs", len(object_ids))

total IDs 10777


## Скачать данные по объектам

In [4]:
objects = []  # Данные объектов.

for object_id in tqdm(object_ids):
    object_id_hash = hashlib.md5(str(object_id).encode("utf-8")).hexdigest()
    object_file_path = f"{BASE_DIR}/objects/{object_id_hash[:2]}/{object_id}.pkl"

    if not os.path.isfile(object_file_path):
        # Скачать данные объекта.
        url = f"https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{object_id}"
        res = requests.get(url)

        if 200 != res.status_code:
            continue

        object_data = res.json()
        objects.append(object_data)

        # Сохранить объект в файл.
        os.makedirs(os.path.dirname(object_file_path), exist_ok=True)

        with open(object_file_path, "wb") as f:
            pickle.dump(object_data, f)
    else:
        # Данные объектов уже есть.
        with open(object_file_path, "rb") as f:
            object_data = pickle.load(f)
            objects.append(object_data)
            
print("total objects:", len(objects))

  0%|          | 0/10777 [00:00<?, ?it/s]

total objects: 10777


## Cохранение информации о всех строящихся объектах

In [5]:
# Сохранить в pandas DataFrame из JSON'а как есть.
df = pd.json_normalize(objects)
df

,errcode,data.id,data.hobjId,data.pdId,data.developer.devId,data.developer.devShortCleanNm,data.developer.devShortNm,data.developer.devFullCleanNm,data.developer.problObjCnt,data.developer.buildObjCnt,...,data.metro.colors,data.airQualityIndexValue,data.transportDistIndex,data.transportDistIndexValue,data.greenAreaIndexValue,data.complexShortNm,data.developer.bankruptStage.bankruptStageCd,data.developer.bankruptStage.bankruptStageDesc,data.developer.bankruptStage.bankruptLawUrl,data.developer.orgBankruptMsgDttm
0,0,32787,43579,181636,6946,ТКС,ТКС,ТАТКАМСТРОЙ,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,32793,43585,162963,6434,СЗ ДЖЕВОССЕТ,СЗ ДЖЕВОССЕТ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК ДЖЕВОССЕТ,0,1,...,[#0072BA],7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,32849,43641,167016,1577,СЗ КАПИТАЛ ГРУПП,СЗ КАПИТАЛ ГРУПП,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИК КАПИТАЛ ГРУПП,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,32892,43687,117906,2551,СЗ МАГИ,СЗ МАГИ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИКМАГИ,0,12,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,32893,43688,117906,2551,СЗ МАГИ,СЗ МАГИ,СПЕЦИАЛИЗИРОВАННЫЙ ЗАСТРОЙЩИКМАГИ,0,12,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10772,0,32607,43387,166974,1195,ИНВЕСТЛЭНД,ИНВЕСТЛЭНД,ИНВЕСТЛЭНД,0,13,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10773,0,32608,43388,166974,1195,ИНВЕСТЛЭНД,ИНВЕСТЛЭНД,ИНВЕСТЛЭНД,0,13,...,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10774,0,32611,43391,169884,7951,ДИПЛОМАТ ПЛЮС,ДИПЛОМАТ ПЛЮС,ДИПЛОМАТ ПЛЮС,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10775,0,32618,43405,152681,2224,КРЕПОСТЬ,КРЕПОСТЬ,КРЕПОСТЬ,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Сохранить в pandas DataFrame из JSON'а выбрав основные поля данных

In [6]:
# Поиск элемента в JSON'е.
def json_search(json_data, key, default=None):
    try:
        for nested_key in key.split("."): 
            json_data = json_data.get(nested_key)
        return json_data
    except AttributeError:
        return default

# Поиск object в JSON'е по его ID.
def find_object_by_id(objects, id):
    for i, object in enumerate(objects):
        if id == json_search(object, "data.id"):
            return i
    raise LookupError(f"Object with ID={id} not found")

In [7]:
# Подготовка данных для сохранения в DataFrame.
tab_dict = defaultdict(list)

# Для отладки
# offset = find_object_by_id(objects, 46687)
# objects = objects[offset:offset+10]

for object in tqdm(objects):
    tab_dict["id"].append(json_search(object, "data.id"))
    
    tab_dict["Застройщик"].append(" ".join([
        json_search(object, "data.developer.orgForm.shortForm"),
        json_search(object, "data.developer.devShortNm")
    ]))
    tab_dict["Группа компаний"].append(json_search(object, "data.developer.developerGroupName"))
    tab_dict["ЖК"].append(json_search(object, "data.nameObj"))
   
    tab_dict["Ввод в эксплуатацию"].append(json_search(object, "data.objReady100PercDt"))
    tab_dict["Выдача ключей"].append(json_search(object, "data.objTransferPlanDt"))
    tab_dict["Средняя цена за 1 м2"].append(json_search(object, "data.objPriceAvg"))
    tab_dict["Распроданность квартир"].append(json_search(object, "data.soldOutPerc"))
    
    tab_dict["Класс энергоэффективности здания"].append(json_search(object, "data.objEnergyEffShortDesc"))
    tab_dict["Генподрядчики"].append(json_search(object, "data.generalContractorNm"))
    
    # Основные характеристики
    tab_dict["Класс недвижимости"].append(json_search(object, "data.objLkClassDesc"))
    tab_dict["Материал стен"].append(json_search(object, "data.wallMaterialShortDesc"))
    tab_dict["Тип отделки"].append(json_search(object, "data.objLkFinishTypeDesc"))
    tab_dict["Свободная планировка"].append(json_search(object, "data.objLkFreePlanDesc"))
    tab_dict["Количество этажей"].append(json_search(object, "data.objFloorCnt"))
    tab_dict["Количество квартир"].append(json_search(object, "data.objElemLivingCnt"))
    tab_dict["Жилая площадь, м2"].append(json_search(object, "data.objSquareLiving"))
    tab_dict["Высота потолков, м"].append(json_search(object, "data.objLivCeilingHeight"))
    
    # Благоустройство двора
    tab_dict["Велосипедные дорожки"].append(json_search(object, "data.objInfrstrBicycleLaneFlg"))
    tab_dict["Количество детских площадок"].append(json_search(object, "data.objInfrstrPlaygrndCnt"))
    tab_dict["Количество спортивных площадок"].append(json_search(object, "data.objInfrstrSportGroundCnt"))
    tab_dict["Количество площадок для сбора мусора"].append(json_search(object, "data.objInfrstrTrashAreaCnt"))
    
    # Парковочное пространство
    tab_dict["Количество мест в паркинге"].append(json_search(object, "data.objElemParkingCnt"))
    tab_dict["Гостевые места на придомовой территории"].append(json_search(object, "data.objInfrstrObjPrkngCnt"))
    tab_dict["Гостевые места вне придомовой территории"].append(json_search(object, "data.objInfrstrNotObjPrkngCnt"))
    
    # Безбарьерная среда
    tab_dict["Наличие пандуса"].append(json_search(object, "data.objInfrstrRampFlg"))
    tab_dict["Наличие понижающих площадок"].append(json_search(object, "data.objInfrstrCurbLoweringFlg"))
    tab_dict["Количество инвалидных подъемников"].append(json_search(object, "data.objElevatorWheelchairCnt"))
    
    # Наличие лифтов в доме
    tab_dict["Количество подъездов"].append(json_search(object, "data.quartography.objLivElemEntrCnt"))
    tab_dict["Количество пассажирских лифтов"].append(json_search(object, "data.objElevatorPassengerCnt"))
    tab_dict["Количество грузовых и грузопассажирских лифтов"].append(json_search(object, "data.objElevatorCargoPassengerCnt"))

  0%|          | 0/10777 [00:00<?, ?it/s]

In [8]:
# Cохранить в DataFrame.
df = pd.DataFrame(tab_dict)
df

,id,Застройщик,Группа компаний,ЖК,Ввод в эксплуатацию,Выдача ключей,Средняя цена за 1 м2,Распроданность квартир,Класс энергоэффективности здания,Генподрядчики,...,Количество площадок для сбора мусора,Количество мест в паркинге,Гостевые места на придомовой территории,Гостевые места вне придомовой территории,Наличие пандуса,Наличие понижающих площадок,Количество инвалидных подъемников,Количество подъездов,Количество пассажирских лифтов,Количество грузовых и грузопассажирских лифтов
0,32787,ООО ТКС,ТАТКАМСТРОЙ,"ЖК ""Современный""",2022-12-31,28-02-2023 12:00,61816.0,1.000000,A,"ООО ""Билант"" (ИНН: 1651000203)",...,1.0,0,89.0,282.0,1,1,0.0,1.0,1.0,1
1,32793,ООО СЗ ДЖЕВОССЕТ,ПроГород (группа ВЭБ.РФ),"Микрогород ""В лесу""",2023-03-31,31-08-2023 12:00,151912.0,0.863037,B,"ООО ""Компания Б.В.С."" (ИНН: 7703543324)",...,1.0,264,139.0,0.0,1,1,0.0,8.0,14.0,22
2,32849,ООО СЗ КАПИТАЛ ГРУПП,Центр Строительных Услуг,"Микрорайон ""Тихий Берег"", 004",2022-12-31,29-12-2023 12:00,51105.0,0.953005,D,"ООО ""Кохмаситистрой"" (ИНН: 3702149712)",...,1.0,0,10.0,10.0,1,1,8.0,5.0,3.0,0
3,32892,ООО СЗ МАГИ,None,"ЖК ""СКАЗКА"" дом 64",2022-12-31,28-02-2023 12:00,160261.0,0.470130,A,ООО Перспектива (ИНН: 7728883629),...,1.0,0,15.0,0.0,0,1,0.0,2.0,2.0,0
4,32893,ООО СЗ МАГИ,None,"ЖК ""СКАЗКА"" дом 65",2022-12-31,28-02-2023 12:00,148909.0,0.648701,A,ООО Перспектива (ИНН: 7728883629),...,1.0,0,15.0,0.0,0,1,0.0,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10772,32607,ООО ИНВЕСТЛЭНД,Villagio Estate,"ЖК Футуро Парк (Futuro Park), Корпус №58",2023-12-31,29-02-2024 12:00,NaN,NaN,B,ООО Проектион (ИНН: 5024207331),...,2.0,0,14.0,30.0,0,1,0.0,2.0,0.0,0
10773,32608,ООО ИНВЕСТЛЭНД,Villagio Estate,"ЖК Футуро Парк (Futuro Park), Корпус №59",2023-12-31,29-02-2024 12:00,NaN,NaN,B,ООО Проектион (ИНН: 5024207331),...,2.0,0,14.0,30.0,0,1,0.0,2.0,0.0,0
10774,32611,ООО ДИПЛОМАТ ПЛЮС,ВнешОптоТорг,"ЖК ""Южное море""",2031-03-31,17-09-2031 12:00,94409.0,1.000000,B,ООО КР (ИНН: 2367000898),...,1.0,0,0.0,0.0,1,1,0.0,1.0,2.0,2
10775,32618,ООО КРЕПОСТЬ,Регион Девелопмент,"Жилой комплекс ""пер.Светлогорский II очередь о...",2025-09-30,31-07-2025 12:00,83647.0,0.146932,A+,None,...,0.0,0,52.0,0.0,0,1,11.0,3.0,3.0,3


In [9]:
# Cохрание DataFrame в Excel.
xlsx_file_path = f"{BASE_DIR}/nashdom.xlsx"

df.to_excel(xlsx_file_path, index=False)

In [10]:
# Загрузка DataFrame из Excel.
df_from_excel = pd.read_excel(xlsx_file_path, index_col=0)
df_from_excel

,Застройщик,Группа компаний,ЖК,Ввод в эксплуатацию,Выдача ключей,Средняя цена за 1 м2,Распроданность квартир,Класс энергоэффективности здания,Генподрядчики,Класс недвижимости,...,Количество площадок для сбора мусора,Количество мест в паркинге,Гостевые места на придомовой территории,Гостевые места вне придомовой территории,Наличие пандуса,Наличие понижающих площадок,Количество инвалидных подъемников,Количество подъездов,Количество пассажирских лифтов,Количество грузовых и грузопассажирских лифтов
id,,,,,,,,,,,,,,,,,,,,,
32787,ООО ТКС,ТАТКАМСТРОЙ,"ЖК ""Современный""",2022-12-31,28-02-2023 12:00,61816.0,1.000000,A,"ООО ""Билант"" (ИНН: 1651000203)",Комфорт,...,1.0,0,89.0,282.0,1,1,0.0,1.0,1.0,1
32793,ООО СЗ ДЖЕВОССЕТ,ПроГород (группа ВЭБ.РФ),"Микрогород ""В лесу""",2023-03-31,31-08-2023 12:00,151912.0,0.863037,B,"ООО ""Компания Б.В.С."" (ИНН: 7703543324)",Комфорт,...,1.0,264,139.0,0.0,1,1,0.0,8.0,14.0,22
32849,ООО СЗ КАПИТАЛ ГРУПП,Центр Строительных Услуг,"Микрорайон ""Тихий Берег"", 004",2022-12-31,29-12-2023 12:00,51105.0,0.953005,D,"ООО ""Кохмаситистрой"" (ИНН: 3702149712)",Комфорт,...,1.0,0,10.0,10.0,1,1,8.0,5.0,3.0,0
32892,ООО СЗ МАГИ,NaN,"ЖК ""СКАЗКА"" дом 64",2022-12-31,28-02-2023 12:00,160261.0,0.470130,A,ООО Перспектива (ИНН: 7728883629),Бизнес,...,1.0,0,15.0,0.0,0,1,0.0,2.0,2.0,0
32893,ООО СЗ МАГИ,NaN,"ЖК ""СКАЗКА"" дом 65",2022-12-31,28-02-2023 12:00,148909.0,0.648701,A,ООО Перспектива (ИНН: 7728883629),Бизнес,...,1.0,0,15.0,0.0,0,1,0.0,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32607,ООО ИНВЕСТЛЭНД,Villagio Estate,"ЖК Футуро Парк (Futuro Park), Корпус №58",2023-12-31,29-02-2024 12:00,NaN,NaN,B,ООО Проектион (ИНН: 5024207331),Бизнес,...,2.0,0,14.0,30.0,0,1,0.0,2.0,0.0,0
32608,ООО ИНВЕСТЛЭНД,Villagio Estate,"ЖК Футуро Парк (Futuro Park), Корпус №59",2023-12-31,29-02-2024 12:00,NaN,NaN,B,ООО Проектион (ИНН: 5024207331),Бизнес,...,2.0,0,14.0,30.0,0,1,0.0,2.0,0.0,0
32611,ООО ДИПЛОМАТ ПЛЮС,ВнешОптоТорг,"ЖК ""Южное море""",2031-03-31,17-09-2031 12:00,94409.0,1.000000,B,ООО КР (ИНН: 2367000898),Бизнес,...,1.0,0,0.0,0.0,1,1,0.0,1.0,2.0,2


In [11]:
# Сохранение DataFrame в pickle.
pkl_file_path = f"{BASE_DIR}/nashdom.pkl"

df.to_pickle(pkl_file_path)

In [12]:
# Загрузка DataFrame из pickle.
df_from_pickle = pd.read_pickle(pkl_file_path)
df_from_pickle

,id,Застройщик,Группа компаний,ЖК,Ввод в эксплуатацию,Выдача ключей,Средняя цена за 1 м2,Распроданность квартир,Класс энергоэффективности здания,Генподрядчики,...,Количество площадок для сбора мусора,Количество мест в паркинге,Гостевые места на придомовой территории,Гостевые места вне придомовой территории,Наличие пандуса,Наличие понижающих площадок,Количество инвалидных подъемников,Количество подъездов,Количество пассажирских лифтов,Количество грузовых и грузопассажирских лифтов
0,32787,ООО ТКС,ТАТКАМСТРОЙ,"ЖК ""Современный""",2022-12-31,28-02-2023 12:00,61816.0,1.000000,A,"ООО ""Билант"" (ИНН: 1651000203)",...,1.0,0,89.0,282.0,1,1,0.0,1.0,1.0,1
1,32793,ООО СЗ ДЖЕВОССЕТ,ПроГород (группа ВЭБ.РФ),"Микрогород ""В лесу""",2023-03-31,31-08-2023 12:00,151912.0,0.863037,B,"ООО ""Компания Б.В.С."" (ИНН: 7703543324)",...,1.0,264,139.0,0.0,1,1,0.0,8.0,14.0,22
2,32849,ООО СЗ КАПИТАЛ ГРУПП,Центр Строительных Услуг,"Микрорайон ""Тихий Берег"", 004",2022-12-31,29-12-2023 12:00,51105.0,0.953005,D,"ООО ""Кохмаситистрой"" (ИНН: 3702149712)",...,1.0,0,10.0,10.0,1,1,8.0,5.0,3.0,0
3,32892,ООО СЗ МАГИ,None,"ЖК ""СКАЗКА"" дом 64",2022-12-31,28-02-2023 12:00,160261.0,0.470130,A,ООО Перспектива (ИНН: 7728883629),...,1.0,0,15.0,0.0,0,1,0.0,2.0,2.0,0
4,32893,ООО СЗ МАГИ,None,"ЖК ""СКАЗКА"" дом 65",2022-12-31,28-02-2023 12:00,148909.0,0.648701,A,ООО Перспектива (ИНН: 7728883629),...,1.0,0,15.0,0.0,0,1,0.0,2.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10772,32607,ООО ИНВЕСТЛЭНД,Villagio Estate,"ЖК Футуро Парк (Futuro Park), Корпус №58",2023-12-31,29-02-2024 12:00,NaN,NaN,B,ООО Проектион (ИНН: 5024207331),...,2.0,0,14.0,30.0,0,1,0.0,2.0,0.0,0
10773,32608,ООО ИНВЕСТЛЭНД,Villagio Estate,"ЖК Футуро Парк (Futuro Park), Корпус №59",2023-12-31,29-02-2024 12:00,NaN,NaN,B,ООО Проектион (ИНН: 5024207331),...,2.0,0,14.0,30.0,0,1,0.0,2.0,0.0,0
10774,32611,ООО ДИПЛОМАТ ПЛЮС,ВнешОптоТорг,"ЖК ""Южное море""",2031-03-31,17-09-2031 12:00,94409.0,1.000000,B,ООО КР (ИНН: 2367000898),...,1.0,0,0.0,0.0,1,1,0.0,1.0,2.0,2
10775,32618,ООО КРЕПОСТЬ,Регион Девелопмент,"Жилой комплекс ""пер.Светлогорский II очередь о...",2025-09-30,31-07-2025 12:00,83647.0,0.146932,A+,None,...,0.0,0,52.0,0.0,0,1,11.0,3.0,3.0,3


In [13]:
# Сохранение DataFrame в БД sqlite3.
db_path = f"{BASE_DIR}/nashdom.sqlite3"
conn = sqlite3.connect(db_path)
df.to_sql("nashdom", conn, if_exists="replace", index = False)
conn.commit()
conn.close()

In [14]:
# Загрузка DataFrame из БД sqlite3.
conn = sqlite3.connect(db_path)
df_sql = pd.read_sql('SELECT * FROM "nashdom" ORDER BY "Застройщик" LIMIT 5', conn)
conn.close()
df_sql

,id,Застройщик,Группа компаний,ЖК,Ввод в эксплуатацию,Выдача ключей,Средняя цена за 1 м2,Распроданность квартир,Класс энергоэффективности здания,Генподрядчики,...,Количество площадок для сбора мусора,Количество мест в паркинге,Гостевые места на придомовой территории,Гостевые места вне придомовой территории,Наличие пандуса,Наличие понижающих площадок,Количество инвалидных подъемников,Количество подъездов,Количество пассажирских лифтов,Количество грузовых и грузопассажирских лифтов
0,13383,АО 2МЕН ГРУПП,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,None,2022-12-31,28-02-2023 12:00,53109.0,0.406985,B,ООО Транспромжилстрой-2005 (ИНН: 7204097730),...,1.0,31,0.0,11.0,1,1,0.0,1.0,5.0,7
1,13391,АО 2МЕН ГРУПП,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,None,2022-12-31,28-02-2023 12:00,55893.0,0.450622,B,ООО Транспромжилстрой-2005 (ИНН: 7204097730),...,1.0,41,0.0,9.0,1,1,0.0,2.0,2.0,6
2,13397,АО 2МЕН ГРУПП,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,None,2024-12-31,28-02-2025 12:00,45523.0,0.999984,B,ООО Транспромжилстрой-2005 (ИНН: 7204097730),...,1.0,0,11.0,10.0,1,1,0.0,4.0,5.0,9
3,13398,АО 2МЕН ГРУПП,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,None,2024-12-31,28-02-2025 12:00,47886.0,1.000000,B,ООО Транспромжилстрой-2005 (ИНН: 7204097730),...,1.0,0,4.0,15.0,1,1,0.0,3.0,3.0,6
4,13399,АО 2МЕН ГРУПП,2МЕН ГРУПП ДЕВЕЛОПМЕНТ,None,2024-12-31,28-02-2025 12:00,50768.0,0.917243,B,ООО Транспромжилстрой-2005 (ИНН: 7204097730),...,1.0,125,0.0,0.0,1,1,0.0,NaN,8.0,15


## Cкачать фотографии для всех строящихся объектов

In [15]:
def download_photos(objects):
    for object in tqdm(objects):
        object_id = json_search(object, "data.id")
        photo_render_dto = json_search(object, "data.photoRenderDTO")

        for photo in photo_render_dto:
            photo_url = photo.get("objRenderPhotoUrl")
            photo_name = photo.get("objRenderPhotoNm")
            object_ready_desc = photo.get("objReadyDesc").lower()  # Статус объекта.
            
            if object_ready_desc == "строится":
                object_file_path = f"{BASE_DIR}/images/{object_id}/{photo_name}"

                if not os.path.isfile(object_file_path):
                    res = requests.get(photo_url)

                    if 200 != res.status_code:
                        continue

                    os.makedirs(os.path.dirname(object_file_path), exist_ok=True)

                    with open(object_file_path, "wb") as f:
                        f.write(res.content)

In [16]:
# Остановил загрузку (очень много файлов).
download_photos(objects)

  0%|          | 0/10777 [00:00<?, ?it/s]

KeyboardInterrupt: 